In [20]:
import os
import getpass
import pandas as pd
import requests
import json
import yarl

In [21]:
CMC_API_KEY =  os.getenv('CMC_API_KEY', getpass.getpass('CoinMarketCap API Key: '))
CMC_URL = yarl.URL('https://pro-api.coinmarketcap.com/v1/')

CoinMarketCap API Key: ········


In [34]:
CURRENCY='USD'

In [25]:
def get_latest_listings(start: int = 1, limit: int = 5000, currency: str = 'USD'):
    """
    Get crypto listings from CoinMarketCap
    """
    url = CMC_URL.join(yarl.URL('cryptocurrency/listings/latest'))
    parameters = {
      'start': start,
      'limit': limit,
      'convert': currency
    }
    headers = {
      'Accepts': 'application/json',
      'X-CMC_PRO_API_KEY': CMC_API_KEY,
    }

    session = requests.Session()
    session.headers.update(headers)

    try:
        response = session.get(str(url), params=parameters)
        response.raise_for_status()
        return json.loads(response.text)
    except Exception as e:
        print(e)

In [26]:
listings = get_latest_listings(currency=CURRENCY)

Get top 10 by market cap excluding stable coins

In [68]:
exclusions = ['usdt', 'usdc', 'busd', 'dai', 'tusd', 'usdp'] # Add..

In [80]:
columns = ['Name', 'Symbol', 'Price', 'Market Cap', 'Number of Tokens to Buy','Cost']
df = pd.DataFrame(columns=columns)

In [81]:
current_idx = 0
while len(df.index) < 10:
    listing = listings['data'][current_idx]
    if listing['symbol'].lower() not in exclusions:
        df = df.append(
            pd.Series(
                [
                    listing['name'],
                    listing['symbol'],
                    listing['quote'][CURRENCY]['price'],
                    listing['quote'][CURRENCY]['market_cap'],
                    'N/A',
                    'N/A'
                ],
                index=columns

            ),
            ignore_index=True
        )
    current_idx += 1

/tmp/ipykernel_29405/2925468603.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(
/tmp/ipykernel_29405/2925468603.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(
/tmp/ipykernel_29405/2925468603.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(
/tmp/ipykernel_29405/2925468603.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(
/tmp/ipykernel_29405/2925468603.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(
/tmp/ipykernel_29405/2925468603.py:5: FutureWarning: Th

In [82]:
df

,Name,Symbol,Price,Market Cap,Number of Tokens to Buy,Cost
0,Bitcoin,BTC,24591.222208,4.701912e+11,N/A,N/A
1,Ethereum,ETH,2001.958214,2.441275e+11,N/A,N/A
2,BNB,BNB,328.649060,5.302334e+10,N/A,N/A
3,Cardano,ADA,0.559625,1.899033e+10,N/A,N/A
4,XRP,XRP,0.381215,1.873262e+10,N/A,N/A
5,Solana,SOL,47.548227,1.658038e+10,N/A,N/A
6,Polkadot,DOT,9.527468,1.054214e+10,N/A,N/A
7,Dogecoin,DOGE,0.073308,9.725833e+09,N/A,N/A
8,Avalanche,AVAX,29.676961,8.456113e+09,N/A,N/A
9,Polygon,MATIC,1.012923,8.139140e+09,N/A,N/A


## Equal Weight Index

In [73]:
portfolio_value = float(input(f'Portfolio value ({CURRENCY}): '))

Portfolio value (USD): 1000


In [ ]:
len(df.index)

In [85]:
position_size = float(portfolio_value) / len(df.index)
for i in range(0, len(df['Symbol'])):
    df.loc[i, 'Number of Tokens to Buy'] = position_size / df['Price'][i]
    df.loc[i, 'Cost'] = position_size
df

,Name,Symbol,Price,Market Cap,Number of Tokens to Buy,Cost
0,Bitcoin,BTC,24591.222208,4.701912e+11,0.004066,100.0
1,Ethereum,ETH,2001.958214,2.441275e+11,0.049951,100.0
2,BNB,BNB,328.649060,5.302334e+10,0.304276,100.0
3,Cardano,ADA,0.559625,1.899033e+10,178.691188,100.0
4,XRP,XRP,0.381215,1.873262e+10,262.319008,100.0
5,Solana,SOL,47.548227,1.658038e+10,2.103128,100.0
6,Polkadot,DOT,9.527468,1.054214e+10,10.495968,100.0
7,Dogecoin,DOGE,0.073308,9.725833e+09,1364.106989,100.0
8,Avalanche,AVAX,29.676961,8.456113e+09,3.369617,100.0
9,Polygon,MATIC,1.012923,8.139140e+09,98.724235,100.0
